# Direct Distances

In this approach, I make the assumption that the distance between two shops is independent of the road network.

## Generate Random Warehouses

In [1]:
import copy
import os
import pickle
import random

from geopy.geocoders import Nominatim
from haversine import Unit, haversine_vector
import pulp as pl

In [2]:
if not os.path.exists("warehouses.pkl"):
    print("Generating warehouses info...")

    warehouses = []
    for i in range(1, 101):
        name = f"Warehouse {i}"
        lat = random.uniform(
            10.668799, 10.878041
        )  # Latitude range of Ho Chi Minh City
        long = random.uniform(
            106.451840, 106.813580
        )  # Longitude range of Ho Chi Minh City
        description = None
        warehouses.append(
            {
                "index": i,
                "name": name,
                "lat": lat,
                "long": long,
                "description": "",
            }
        )

    geolocator = Nominatim(user_agent="bk-imp")

    for i in range(len(warehouses)):
        lat, long = warehouses[i]["lat"], warehouses[i]["long"]
        location = geolocator.reverse(f"{lat}, {long}")
        address = location.address
        warehouses[i]["description"] = address

    with open("warehouses.pkl", "wb") as f:
        # Shuffle warehouses to generate random order
        random.shuffle(warehouses)
        pickle.dump(warehouses, f)
else:
    with open("warehouses.pkl", "rb") as f:
        warehouses = pickle.load(f)

In [3]:
with open("stores.pkl", "rb") as f:
    stores = pickle.load(f)

## Down Sample

In [4]:
warehouses = random.choices(warehouses, k=10)
stores = random.choices(stores, k=50)

## Calculate Distance Matrix

In [5]:
warehouses_stores_dinstance_matrix = haversine_vector(
    [(store["lat"], store["long"]) for store in stores],
    [(warehouse["lat"], warehouse["long"]) for warehouse in warehouses],
    Unit.KILOMETERS,
    comb=True,
)

In [6]:
warehouses_stores_dinstance_matrix.shape

(10, 50)

## Linear Programming

In [7]:
# Set up the problem
problem = pl.LpProblem("Warehouse_Optimization", pl.LpMinimize)

### Define the environment

In [9]:
M = 10 # Number of warehouses
N = 50 # Number of stores
warehouses = range(M)
w_cost = random.sample(range(100, 201), M)  # Operational cost of warehouses
w_capacity = random.sample(range(1800, 2001), M)  # Operational cost of warehouses
stores = range(N)
demands = random.sample(range(10, 521), N)  # Daily demand of stores
d_cost = copy.copy(
    warehouses_stores_dinstance_matrix
)  # Delivery cost between warehouses vs stores

In [10]:
assert sum(w_capacity) > sum(demands)

### Define the decision variables

In [11]:
wr = pl.LpVariable.dicts(
    "warehouse_store_connection", (warehouses, stores), cat="Binary"
)

### Define the objective function

In [12]:
total_cost = pl.lpSum(w_cost[w] * wr[w][s] + d_cost[w][s] * wr[w][s] for w in warehouses for s in stores)
problem += total_cost

### Define the constraints

In [13]:
# Each store must be assigned to one warehouse
for i in stores:
    problem += pl.lpSum(wr[j][i] for j in warehouses) == 1

In [14]:
# Total demand of assigned stores for a warehouse must be smaller than its capacity
for w in warehouses:
    problem += pl.lpSum(demands[s] * wr[w][s] for s in stores) <= w_capacity[w]

## Solve the problem!!!

In [15]:
problem.solve(pl.PULP_CBC_CMD(maxSeconds=600))

/home/terrabot/bk-imp/.venv/lib/python3.10/site-packages/pulp/apis/coin_api.py:95: UserWarning: Parameter maxSeconds is being depreciated for timeLimit
  warnings.warn("Parameter maxSeconds is being depreciated for timeLimit")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/terrabot/bk-imp/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/6c39bfc9c3f74c6cae188360d98eb8c1-pulp.mps sec 600 timeMode elapsed branch printingOptions all solution /tmp/6c39bfc9c3f74c6cae188360d98eb8c1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 65 COLUMNS
At line 2566 RHS
At line 2627 BOUNDS
At line 3128 ENDATA
Problem MODEL has 60 rows, 500 columns and 1000 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 600
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5834.87 - 0.00 seconds
Cgl0005I 50 SOS with 500 members
Cgl0004I processed model has 60 rows, 500 columns (500 integer (500 of which binary)) and 1000 elements
Cbc0038I Initial state - 12 integers unsatisfied sum - 3.48125
Cbc0038I Pass   1: suminf.    0.72234 (3) obj. 5880.35 iterations 20
Cbc0038I Solution found

1

In [19]:
# Print the optimal solution
print("Optimal Solution:")
for w in warehouses:
    assigned_stores = []
    for s in stores:
        if pl.value(wr[w][s]) == 1:
            assigned_stores.append(s)
    print(f"Warehouse {w} serves these stores:", assigned_stores)

# Print the total cost
print("Total Cost: ", pl.value(problem.objective))

Optimal Solution:
Warehouse 0 serves these stores: [3, 6, 7, 11, 18, 19, 22, 26, 28, 33, 34, 49]
Warehouse 1 serves these stores: [0, 2, 14, 16, 20, 21, 35, 36, 37, 39, 43]
Warehouse 2 serves these stores: []
Warehouse 3 serves these stores: []
Warehouse 4 serves these stores: [27, 30, 31, 38, 45]
Warehouse 5 serves these stores: [5, 12, 15, 23, 29]
Warehouse 6 serves these stores: [46, 47]
Warehouse 7 serves these stores: []
Warehouse 8 serves these stores: [1, 9, 13, 17, 24, 25, 32, 40, 44, 48]
Warehouse 9 serves these stores: [4, 8, 10, 41, 42]
Total Cost:  5849.875433284815


### Gurobi Solver
https://ca.cs.uni-bonn.de/doku.php?id=tutorial:gurobi-install